In [ ]:
import os
import sys
import scipy.io as spio
import numpy as np
import plotly.graph_objects as pgo
import re

sys.path.append("../")
from helpers.get_density import get_density
from helpers.get_spins import get_spins
from helpers.downscale import downscale_3d
from plot_multiple_3Ds import plot_multiple_3Ds
from helpers.grids import get_kw_square_nonzero_grid, get_radius_grid, get_space_grid

In [ ]:
box_N = 128
box_L = 100.0
box_dx = box_L / box_N
def verify_box_spec(sim_path):
	config_path = os.path.join(sim_path, "simConfig.mat")
	simConfig = spio.loadmat(config_path)["simConfig"]
	if box_L != float(simConfig["Lbox"][0, 0][0, 0]):
		raise Exception(ValueError())
	if box_N != int(simConfig["N"][0, 0][0, 0]):
		raise Exception(ValueError())


VOL_SCALE_FACTOR = 4
vol_grid_axis = np.linspace((-box_L + box_dx) / 2, (box_L - box_dx) / 2, num=box_N//VOL_SCALE_FACTOR)
(GX, GY, GZ) = [gi.flatten() for gi in np.meshgrid(vol_grid_axis, vol_grid_axis, vol_grid_axis, indexing='ij')]
	

In [ ]:
def load_snaps_at(sim_path, progress_indices):
	snap_paths = get_snap_paths(sim_path)
	outs = []
	for pr in progress_indices:
		ind = int(pr * (len(snap_paths) - 1))
		snap_path = snap_paths[ind]
		outs.append(load_Psi(snap_path))
	return outs


In [ ]:
def load_Psi(snap_path):
	print(f"loading {snap_path}")
	snap = spio.loadmat(snap_path)
	Psi = np.stack([snap["Psi"][0, i] for i in range(3)], axis=0)
	print(f"loaded")
	return Psi

def get_snap_paths(sim_path):
	verify_box_spec(sim_path)
	sim_files = os.listdir(sim_path)
	searcher = r"snap-Psi-(\d+)-(\d+\.\d*)\.mat"
	snap_files = [re.search(searcher, fname) for fname in sim_files]
	snap_files = [(int(f.groups()[0]), float(f.groups()[1])) for f in snap_files if not(f is None)]
	snap_files = sorted(snap_files)
	snap_files = [f"snap-Psi-{iter}-{time:.2f}.mat" for (iter, time) in snap_files]
	snap_paths = [os.path.join(sim_path, f) for f in snap_files]
	return snap_paths


In [ ]:
def plot_snap_density(Psi):
	Rho = get_density(Psi)
    
	Val = downscale_3d(Rho, VOL_SCALE_FACTOR)
	plot = pgo.Volume(
		x = GX,
		y = GY,
		z = GZ,
		value = np.log1p(Val).flatten() / 3,
		opacity = 0.02,
		surface_count = 128,
		coloraxis = "coloraxis"
	)
	return plot

In [ ]:
import pyfftw.interfaces.numpy_fft as npfft
kw_sq_nz = get_kw_square_nonzero_grid(box_N, box_dx)
import pyfftw
# 128
wisdom = (b'(fftw-3.2.1 fftw_wisdom\n  (fftw_dft_vrank_geq1_register 0 #x10048 #x10048 #x0 #x6d589bdb #xb6b7eea1 #xb0d02405 #x108ac2a7)\n  (fftw_codelet_t1_8 0 #x10048 #x10048 #x0 #x8c0a5bac #xc7d98f75 #x9d5b6d3e #x6437335b)\n  (fftw_dft_rank_geq2_register 0 #x10048 #x10048 #x0 #xb630c759 #xc6c22e12 #xd68f3083 #xf27a1f56)\n  (fftw_codelet_n1_16 0 #x10048 #x10048 #x0 #x5a1705bf #x175ae769 #xcbf107de #xdce7c151)\n  (fftw_codelet_t1_16 0 #x10048 #x10048 #x0 #x918d40ba #x2f955ee0 #xe7c4c97c #xbfa76da6)\n  (fftw_dft_vrank_geq1_register 1 #x10048 #x10048 #x0 #xbd23b190 #x786755ae #x01606f96 #x447c5db2)\n  (fftw_dft_buffered_register 1 #x10048 #x10048 #x0 #x964bee43 #x8e2f2eb1 #x39541446 #xfd6eb35f)\n  (fftw_dft_vrank_geq1_register 0 #x11048 #x11048 #x0 #xa8b52bf0 #x889e143b #xb16b01fe #x7226b0de)\n  (fftw_dft_vrank_geq1_register 0 #x11048 #x11048 #x0 #x4deb8d6f #xc6de29d8 #xd433dcd1 #xdf705ae0)\n  (fftw_dft_thr_vrank_geq1_register 0 #x10048 #x10048 #x0 #x1973e472 #xb98eac30 #x1165313d #x70a826fe)\n  (fftw_codelet_t1_2 0 #x10048 #x10048 #x0 #x58441fd7 #x4fe9323d #xdc7eb606 #x897400b6)\n  (fftw_rdft_rank0_register 3 #x10048 #x10048 #x0 #x224d8f8e #x026a8317 #x9cd4fecd #xa7fef68b)\n  (fftw_codelet_n1_16 0 #x10048 #x10048 #x0 #x12b00dc7 #xa24e39b3 #xd66c41fa #xaf13cf2b)\n  (fftw_dft_buffered_register 1 #x11048 #x11048 #x0 #x8d444846 #xcd9199a7 #xec891840 #xcc7d280f)\n  (fftw_dft_vrank_geq1_register 0 #x10048 #x10048 #x0 #xf33a86f9 #x9b5398d5 #x163933d6 #x5b248e5a)\n  (fftw_dft_thr_vrank_geq1_register 0 #x11048 #x11048 #x0 #xb3361edb #x120ade4d #x04f1b35b #x8286ba8f)\n  (fftw_codelet_t1_16 0 #x10048 #x10048 #x0 #x0222ec4a #x85a4f2f3 #x555a8853 #x785cdd16)\n  (fftw_rdft_rank0_register 3 #x10048 #x10048 #x0 #x7997dbf0 #x92d87f23 #xf2f03278 #x19a08c45)\n  (fftw_rdft_rank0_register 3 #x11048 #x11048 #x0 #x888673c1 #xf41dea05 #x435028a2 #x17dd4157)\n  (fftw_dft_r2hc_register 0 #x11048 #x11048 #x0 #x3c327a3d #x0285b9f6 #x480d976d #xdb56a2ba)\n  (fftw_dft_rank_geq2_register 0 #x11048 #x11048 #x0 #x0438b08b #x7461e111 #x52bbc27b #xc660890d)\n  (fftw_dft_vrank_geq1_register 0 #x10048 #x10048 #x0 #x35f3cde5 #xd711cbd2 #x6b48b526 #x0e72ba62)\n  (fftw_dft_vrank_geq1_register 0 #x10048 #x10048 #x0 #xa8b52bf0 #x889e143b #xb16b01fe #x7226b0de)\n  (fftw_codelet_n1_8 0 #x10048 #x10048 #x0 #x355c0bc0 #xfc7e31f9 #x4547291a #x2cea6c7a)\n  (fftw_codelet_t1_8 0 #x11048 #x11048 #x0 #x75084416 #x0ff9197d #x2d7b7202 #x350e0e36)\n  (fftw_codelet_t1_8 0 #x10048 #x10048 #x0 #x75084416 #x0ff9197d #x2d7b7202 #x350e0e36)\n  (fftw_dft_vrank_geq1_register 0 #x10048 #x10048 #x0 #x6c9d9de9 #xe709d7bb #x38a601b3 #x2b11ce1b)\n  (fftw_codelet_t1_16 0 #x10048 #x10048 #x0 #xf0fa1f28 #xfb8a4b25 #x698592e1 #xd70bb2d1)\n  (fftw_dft_r2hc_register 0 #x10048 #x10048 #x0 #xbe6a74f4 #x794ec416 #xc1f752ea #x92b47c1b)\n  (fftw_dft_nop_register 0 #x11048 #x11048 #x0 #xe61c32a3 #xc7da5a95 #x804f96ae #xa65e1015)\n  (fftw_dft_thr_vrank_geq1_register 0 #x10048 #x10048 #x0 #x6212e7bf #xea1e5221 #x33c1c98a #x98926d0d)\n  (fftw_dft_rank_geq2_register 0 #x10048 #x10048 #x0 #x99efdc69 #x3b2e9a66 #x85e12b67 #x67820348)\n  (fftw_codelet_t1_8 0 #x10048 #x10048 #x0 #x9c7bffee #x2357d5c7 #x654db55b #x2a5ac485)\n  (fftw_dft_thr_vrank_geq1_register 0 #x10048 #x10048 #x0 #xb3361edb #x120ade4d #x04f1b35b #x8286ba8f)\n  (fftw_dft_thr_vrank_geq1_register 0 #x11048 #x11048 #x0 #x43a4f0e3 #x694e4f75 #xa588d78b #x7cfda79f)\n  (fftw_dft_rank_geq2_register 0 #x10048 #x10048 #x0 #xce857e1e #x5936a975 #xa65e43c2 #xaad1f815)\n  (fftw_codelet_n1_8 0 #x10048 #x10048 #x0 #xaefce48e #xdf218aaf #x5c2c9aca #xb557febc)\n  (fftw_dft_vrank_geq1_register 0 #x10048 #x10048 #x0 #x1631498c #xb82ef852 #xae3b1e64 #x3a6585df)\n  (fftw_dft_vrank_geq1_register 0 #x10048 #x10048 #x0 #x5e2fedba #xd704c5dc #xf98faaf9 #xedc703ce)\n  (fftw_dft_vrank_geq1_register 0 #x10048 #x10048 #x0 #x13b995e7 #x49c3d6c4 #xffdcd0c1 #xb8c57ea6)\n  (fftw_dft_buffered_register 1 #x10048 #x10048 #x0 #x6fafb889 #x54a8a6b7 #x9961787d #x6a5d4b13)\n  (fftw_dft_vrank_geq1_register 0 #x10048 #x10048 #x0 #xb04a3ce3 #x11960abe #x052b3018 #x5fc0b3c4)\n  (fftw_dft_thr_vrank_geq1_register 0 #x10048 #x10048 #x0 #xd7467645 #xd07e5d7e #xa431e640 #x8313174b)\n  (fftw_codelet_t1_8 0 #x10048 #x10048 #x0 #x1b93cb4b #x4b6c2216 #xdee21743 #xb0fa5fdd)\n  (fftw_dft_vrank_geq1_register 0 #x10048 #x10048 #x0 #x4623c9fd #x47385273 #xb01594ee #x94fc510b)\n  (fftw_dft_vrank_geq1_register 0 #x10048 #x10048 #x0 #xd2812872 #xab14ef79 #xe41640f9 #x1d188382)\n  (fftw_codelet_n1_8 0 #x10048 #x10048 #x0 #x81513c62 #x0415a00a #x2709a7d4 #xea62b887)\n  (fftw_dft_thr_vrank_geq1_register 0 #x10048 #x10048 #x0 #x80cc33f6 #x6deec2f3 #xd6d451ab #x4303f481)\n  (fftw_dft_vrank_geq1_register 0 #x10048 #x10048 #x0 #x30f485dc #xbab50549 #x9c5912c2 #xa042b265)\n  (fftw_dft_r2hc_register 0 #x10048 #x10048 #x0 #xad1ad47c #x556889f0 #x00c1940b #x577ba4b1)\n  (fftw_dft_r2hc_register 0 #x10048 #x10048 #x0 #x83af23f7 #xf1ecf157 #x2750d0a8 #x63752149)\n  (fftw_codelet_t1_16 0 #x10048 #x10048 #x0 #x0d72d4f0 #xeae93ee8 #x53c7b117 #x608bf83c)\n  (fftw_rdft_rank0_register 4 #x11048 #x11048 #x0 #x6cf7a6dd #x040cd29c #x0a3384dc #x6853753f)\n  (fftw_dft_r2hc_register 0 #x10048 #x10048 #x0 #x809136a4 #x647e19b6 #x5296418d #xb00f7e1b)\n  (fftw_dft_nop_register 0 #x10048 #x10048 #x0 #xe61c32a3 #xc7da5a95 #x804f96ae #xa65e1015)\n  (fftw_dft_vrank_geq1_register 0 #x10048 #x10048 #x0 #xa8557962 #x04945773 #x19c6cdf6 #x31287589)\n  (fftw_codelet_n1_8 0 #x10048 #x10048 #x0 #x9773499c #xe1336692 #x8136d332 #xb3dab729)\n  (fftw_dft_r2hc_register 0 #x11048 #x11048 #x0 #x752c4d71 #xf336a6e8 #xc6a3c4ae #xa302b191)\n  (fftw_dft_r2hc_register 0 #x10048 #x10048 #x0 #xddac6ea6 #x6440ab19 #x524840f2 #xf5b2e5b3)\n  (fftw_dft_buffered_register 1 #x10048 #x10048 #x0 #x8d444846 #xcd9199a7 #xec891840 #xcc7d280f)\n  (fftw_codelet_n1_8 0 #x10048 #x10048 #x0 #x0ae62870 #xfbbe6e54 #x3d2b70ea #x70b328b0)\n  (fftw_dft_r2hc_register 0 #x10048 #x10048 #x0 #x3c327a3d #x0285b9f6 #x480d976d #xdb56a2ba)\n  (fftw_rdft_rank0_register 3 #x10048 #x10048 #x0 #x888673c1 #xf41dea05 #x435028a2 #x17dd4157)\n  (fftw_dft_indirect_transpose_register 0 #x11048 #x11048 #x0 #x494c4971 #xd74ee6a8 #x5e20f34a #x246319b0)\n  (fftw_rdft_rank0_register 4 #x10048 #x10048 #x0 #x6cf7a6dd #x040cd29c #x0a3384dc #x6853753f)\n  (fftw_dft_buffered_register 1 #x11048 #x11048 #x0 #x8f4f806b #x778c3944 #x2cb2e638 #x55eca843)\n  (fftw_dft_buffered_register 0 #x10048 #x10048 #x0 #x63313023 #x0a067517 #xe6e1bb1a #xcd43a17b)\n  (fftw_codelet_n1_8 0 #x10048 #x10048 #x0 #xb0343399 #x70af39ba #x907800b1 #x56ea8e24)\n  (fftw_codelet_t1_16 0 #x10048 #x10048 #x0 #xe8f8158b #xc793e93a #x6e3b876d #xb6737bb3)\n  (fftw_rdft_rank0_register 3 #x10048 #x10048 #x0 #xef7ef064 #x01e5bd0d #xb6e06ca1 #xbf057687)\n  (fftw_dft_r2hc_register 0 #x10048 #x10048 #x0 #xbf9956c8 #xdb60f5b7 #xc7c15a89 #x347aa24c)\n  (fftw_dft_buffered_register 1 #x11048 #x11048 #x0 #x6fafb889 #x54a8a6b7 #x9961787d #x6a5d4b13)\n  (fftw_dft_r2hc_register 0 #x10048 #x10048 #x0 #xfff02cb4 #x513a99fc #xb2b0c6b6 #x9b59c2d3)\n  (fftw_codelet_n1_8 0 #x10048 #x10048 #x0 #x63a3f645 #xc77254a4 #x492a8d53 #xf9c81344)\n  (fftw_dft_nop_register 0 #x10048 #x10048 #x0 #x73ec5be5 #x4f15fdbe #xf360326c #x86d48736)\n  (fftw_dft_buffered_register 1 #x11048 #x11048 #x0 #xda25760a #x37031a7a #x9cac6243 #x095a620e)\n  (fftw_dft_nop_register 0 #x11048 #x11048 #x0 #x244bfe1d #xa74d2e95 #x6b68ae51 #x24b7c486)\n  (fftw_dft_vrank_geq1_register 1 #x11048 #x11048 #x0 #x6c9d9de9 #xe709d7bb #x38a601b3 #x2b11ce1b)\n  (fftw_dft_r2hc_register 0 #x11048 #x11048 #x0 #x8204839f #xb38caae7 #xcd1e17ed #x31e2918d)\n  (fftw_dft_buffered_register 1 #x10048 #x10048 #x0 #x8f4f806b #x778c3944 #x2cb2e638 #x55eca843)\n  (fftw_codelet_t1_16 0 #x10048 #x10048 #x0 #xb57bcea0 #xd232b352 #xa3f5defd #x4e1dbdce)\n  (fftw_dft_vrank_geq1_register 0 #x10048 #x10048 #x0 #x8c978c56 #x1ceac8c9 #x8eb30eac #xc3ff262c)\n  (fftw_dft_vrank_geq1_register 0 #x10048 #x10048 #x0 #x31d70b11 #x4041668c #xaf3dae4b #x76c5acc4)\n  (fftw_rdft_rank0_register 3 #x11048 #x11048 #x0 #x7997dbf0 #x92d87f23 #xf2f03278 #x19a08c45)\n  (fftw_dft_buffered_register 1 #x10048 #x10048 #x0 #xc910137f #x256d842e #xd0146f87 #xf9b5c645)\n  (fftw_codelet_t1_16 0 #x10048 #x10048 #x0 #xc18c8c55 #x5781adf5 #x193c2775 #x0aa197d2)\n  (fftw_dft_thr_vrank_geq1_register 0 #x10048 #x10048 #x0 #x43a4f0e3 #x694e4f75 #xa588d78b #x7cfda79f)\n  (fftw_rdft_rank0_register 3 #x10048 #x10048 #x0 #xcc6b0242 #x3c5f8af6 #xcf57ee5f #x710ee5d1)\n  (fftw_codelet_t1_16 0 #x10048 #x10048 #x0 #x93f293d3 #xaa3fa02d #xc1bb2fb3 #xc7f29cae)\n  (fftw_codelet_n1_8 0 #x10048 #x10048 #x0 #x558f74d6 #x794d5490 #xa248cff3 #x946b61ec)\n  (fftw_dft_buffered_register 1 #x10048 #x10048 #x0 #x2f20bee0 #xbd762640 #x01cb2f31 #xb0e59e71)\n  (fftw_codelet_n1_8 0 #x10048 #x10048 #x0 #x139ab3dd #x0ea6727a #x659ba669 #xe549c25b)\n  (fftw_dft_vrank_geq1_register 0 #x10048 #x10048 #x0 #x7b89f60a #x48f6722d #x453ffb0b #x6c8610c6)\n  (fftw_dft_buffered_register 1 #x10048 #x10048 #x0 #x8fdde13b #xa9705b4d #xab98d3d0 #x8c7d03b1)\n  (fftw_dft_buffered_register 0 #x10048 #x10048 #x0 #x494c4971 #xd74ee6a8 #x5e20f34a #x246319b0)\n  (fftw_dft_buffered_register 1 #x10048 #x10048 #x0 #x4d0fec7b #xfc47ad10 #x72f71023 #xd1d09124)\n  (fftw_dft_thr_vrank_geq1_register 0 #x10048 #x10048 #x0 #x49dbd2d4 #x55318646 #xea1470bb #x83f3b48e)\n  (fftw_dft_vrank_geq1_register 0 #x10048 #x10048 #x0 #x4deb8d6f #xc6de29d8 #xd433dcd1 #xdf705ae0)\n  (fftw_codelet_n1_8 0 #x10048 #x10048 #x0 #xf4ea3bda #x107f00a1 #x5dc624b3 #x270a45fa)\n  (fftw_dft_r2hc_register 0 #x11048 #x11048 #x0 #x83af23f7 #xf1ecf157 #x2750d0a8 #x63752149)\n  (fftw_dft_buffered_register 1 #x10048 #x10048 #x0 #x89e881e6 #xa9c91f6b #x2bbde60e #xea5dac71)\n  (fftw_dft_r2hc_register 0 #x10048 #x10048 #x0 #xf2d7e86c #xb351e502 #xc5846248 #xca6776c1)\n  (fftw_dft_nop_register 0 #x11048 #x11048 #x0 #x73ec5be5 #x4f15fdbe #xf360326c #x86d48736)\n  (fftw_dft_nop_register 0 #x10048 #x10048 #x0 #x8118f2dd #x12edb6e7 #xc2d13d1c #x433d226b)\n  (fftw_dft_rank_geq2_register 0 #x11048 #x11048 #x0 #xce857e1e #x5936a975 #xa65e43c2 #xaad1f815)\n  (fftw_dft_r2hc_register 0 #x11048 #x11048 #x0 #x8ea336ff #x7c4e2545 #x7dab4916 #x5a2cd4d1)\n  (fftw_dft_vrank_geq1_register 0 #x10048 #x10048 #x0 #xa2169707 #xab0005db #x21abcb3d #x659680d1)\n  (fftw_dft_r2hc_register 0 #x10048 #x10048 #x0 #x752c4d71 #xf336a6e8 #xc6a3c4ae #xa302b191)\n  (fftw_dft_vrank_geq1_register 0 #x10048 #x10048 #x0 #xaa4cc843 #x78946bf1 #x35b84b46 #x687dc117)\n  (fftw_dft_nop_register 0 #x10048 #x10048 #x0 #xd8089e3f #x88b41ff4 #x11001e2d #x427bdb6c)\n  (fftw_codelet_n1_16 0 #x10048 #x10048 #x0 #x8ab20ccf #x7009e799 #x1c9ae857 #x6cbb1487)\n  (fftw_dft_rank_geq2_register 0 #x10048 #x10048 #x0 #x7a7abde1 #xd7a67c1b #x5b67badc #x105a839b)\n  (fftw_dft_vrank_geq1_register 0 #x11048 #x11048 #x0 #xa8557962 #x04945773 #x19c6cdf6 #x31287589)\n  (fftw_dft_vrank_geq1_register 0 #x10048 #x10048 #x0 #x265c8be9 #xc91c7a96 #x69c8bcba #x822ed0df)\n  (fftw_dft_rank_geq2_register 0 #x10048 #x10048 #x0 #x0438b08b #x7461e111 #x52bbc27b #xc660890d)\n  (fftw_codelet_t1_16 0 #x10048 #x10048 #x0 #xe544c9c1 #x572357d4 #xc1de0d2e #xd3e2be3e)\n  (fftw_codelet_n1_16 0 #x11048 #x11048 #x0 #xdecd6bc1 #x1a750077 #x710dda8f #xc684751e)\n  (fftw_rdft_rank0_register 3 #x11048 #x11048 #x0 #x5a8ff7f9 #x01c684a5 #x9a52d8e5 #x3003a2ca)\n  (fftw_rdft_rank0_register 6 #x11048 #x11048 #x0 #x4bb2fd4c #xf7fa03d7 #x259cc9a3 #xd57a9520)\n)\n',
          b'(fftw-3.2.1 fftwf_wisdom\n)\n',
          b'(fftw-3.2.1 fftwl_wisdom\n)\n')
pyfftw.import_wisdom(wisdom)

def get_normalized_grav_potential(Rho):
	V_grav = npfft.fftn(-Rho)
	V_grav = V_grav / kw_sq_nz
	V_grav = np.real(npfft.ifftn(V_grav))
	min_V = np.min(V_grav)
	max_V = np.max(V_grav)
	return (V_grav - min_V) / (max_V - min_V)

def plot_snap_V_grav(Psi):
	Rho = get_density(Psi)
	V_grav = get_normalized_grav_potential(Rho)
	return pgo.Volume(
		x = GX, y = GY, z = GZ,
		value = 1 - downscale_3d(V_grav, VOL_SCALE_FACTOR).flatten(),
		opacity = 0.02,
		surface_count = 128,
		coloraxis = "coloraxis"
	)

In [ ]:
def plot_snap_spins(Psi):
	Rho = get_density(Psi)
	Spins = get_spins(Psi)
	rt3 = 3**0.5
	Spp = [downscale_3d(Spin / Rho, VOL_SCALE_FACTOR).flatten() / rt3 for Spin in Spins]
	return pgo.Cone(
		x=GX, y=GY, z=GZ,
		u=Spp[0], v=Spp[1], w=Spp[2],
		coloraxis = "coloraxis"
	)

In [ ]:
Psis = load_snaps_at(f"../outputs/2022-07-30/8-solitons-random-128-attractive-run-1/", [0.0, 0.5, 1.0])
plots = [
	[plot_snap_density(Psi) for Psi in Psis],
	[plot_snap_V_grav(Psi) for Psi in Psis],
	[plot_snap_spins(Psi) for Psi in Psis]
]
fig, js = plot_multiple_3Ds(plots)
fig.update_layout(
	coloraxis = {
		'colorscale': 'jet',
		'cmin': 0.0,
		'cmax': 1.0,
	},
)
fig.write_html("plot_Rho_Spin_2022-07-30-run-1.html", post_script=js)

In [ ]:
NUM_SIMULATIONS = 12
PROGRESS_STEPS = [9/11, 10/11, 11/11]
INTR_MODES = [
	# filename,		 label,				 color
	("nosi",		"No SI",			"#1abc9c"),
	("attractive",	"Attractive SI",	"#3498db"),
	("repulsive",	"Repulsive SI",		"#f39c12"),
]
all_Psis = [[load_snaps_at(f"../out_remote/2022-07-30/8-solitons-random-128-{filename}-run-{i}/", PROGRESS_STEPS) for (filename, _, _) in INTR_MODES] for i in range(1, 1+NUM_SIMULATIONS)]

In [ ]:
NUM_SIMULATIONS = 2
PROGRESS_STEPS = [v / 18 for v in range(11, 19)]

all_Psis_new = [[load_snaps_at(f"../outputs/2022-08-09/8-solitons-random-128-{filename}-run-{i}/", PROGRESS_STEPS) for (filename, _, _) in INTR_MODES] for i in range(1, 1+NUM_SIMULATIONS)]

In [ ]:
# all_Psis[sim number][interaction mode][progress step][component]

In [ ]:
from plotly.subplots import make_subplots
from tqdm import tqdm
def make_plots(plot_func):
	fig = make_subplots(NUM_SIMULATIONS, len(PROGRESS_STEPS), shared_xaxes=True, shared_yaxes=True)
	for (simul, sim_Psis) in tqdm(enumerate(all_Psis, start=1)):
		for (intr_idx, sim_intr_Psis) in enumerate(sim_Psis):
			for (pro, step_Psi) in enumerate(sim_intr_Psis, start=1):
				trace = plot_func(step_Psi)
				(_, label, color) = INTR_MODES[intr_idx]
				trace.name = label
				trace.line.color = color
				fig.add_trace(trace, simul, pro)

	fig = pgo.Figure(fig)
	return fig

NUM_BINS = 192

In [ ]:
def plot_Rho_over_AvgRho_low(Psi):
	Rho = get_density(Psi)
	Values = Rho / np.average(Rho)
	counts, edges = np.histogram(Values, bins = NUM_BINS, range = (0.0, 1.0), density = True)
	return pgo.Scatter(x=edges, y=counts, mode="lines+markers")
def plot_Rho_over_AvgRho_high(Psi):
	Rho = get_density(Psi)
	Values = Rho / np.average(Rho)
	max_value = np.max(Values)
	counts, edges = np.histogram(Values, bins = np.logspace(0, np.log10(max_value), num = NUM_BINS, base=10.0), range = (1.0, max_value), density = True)
	return pgo.Scatter(x=edges, y=counts, mode="lines+markers")

fig = make_plots(plot_Rho_over_AvgRho_low)
fig.update_yaxes(type = "log")
fig.write_html("2022-07-30_Rho_over_AvgRho_low.html")
fig = make_plots(plot_Rho_over_AvgRho_high)
fig.update_yaxes(type = "log")
fig.update_xaxes(type = "log")
fig.write_html("2022-07-30_Rho_over_AvgRho_high.html")

In [ ]:
def plot_Spin_over_Rho(Psi):
	Rho = get_density(Psi)
	Spins = get_spins(Psi)
	Norms = np.sqrt(np.sum(np.square(Spins), axis=0))
	Values = Norms / Rho
	counts, edges = np.histogram(Values, bins = NUM_BINS, density = True)
	return pgo.Scatter(x=edges, y=counts, mode="lines+markers")

fig = make_plots(plot_Spin_over_Rho)
fig.write_html("2022-07-30_Spin_over_Rho.html")

In [ ]:
def plot_Spin_over_AvgSpin_low(Psi):
	Spins = get_spins(Psi)
	Norms = np.sqrt(np.sum(np.square(Spins), axis=0))
	Values = Norms / np.average(Norms)
	counts, edges = np.histogram(Values, bins = NUM_BINS, range = (0.0, 1.0), density = True)
	return pgo.Scatter(x=edges, y=counts, mode="lines+markers")
def plot_Spin_over_AvgSpin_high(Psi):
	Spins = get_spins(Psi)
	Norms = np.sqrt(np.sum(np.square(Spins), axis=0))
	Values = Norms / np.average(Norms)
	max_value = np.max(Values)
	counts, edges = np.histogram(Values, bins = np.logspace(0, np.log10(max_value), num = NUM_BINS, base=10.0), range = (1.0, max_value), density = True)
	return pgo.Scatter(x=edges, y=counts, mode="lines+markers")

fig = make_plots(plot_Spin_over_AvgSpin_low)
fig.update_yaxes(type = "log")
fig.write_html("2022-07-30_Spin_over_AvgSpin_low.html")
fig = make_plots(plot_Spin_over_AvgSpin_high)
fig.update_yaxes(type = "log")
fig.update_xaxes(type = "log")
fig.write_html("2022-07-30_Spin_over_AvgSpin_high.html")

In [ ]:
MOVING_AVERAGE_WINDOW = 128
Space3 = get_space_grid(box_N, box_dx)
def find_dense_circle(Psi3, fixed_radius: float = None, use_gp = False):
    Rho = get_density(Psi3)
    Rho_f = Rho.flatten()
    Grav = get_normalized_grav_potential(Rho)
    Grav_f = Grav.flatten()
    if use_gp:
        densest_point_idx = np.unravel_index(np.argmin(Grav_f), shape=Grav.shape)
    else:
        densest_point_idx = np.unravel_index(np.argmax(Rho_f), shape=Rho.shape)
    densest_point = tuple([sp[densest_point_idx] for sp in Space3])
    radius = get_radius_grid(box_N, box_dx, densest_point)
    radius_f = radius.flatten()
    sorter = np.argsort(radius_f)
    radius_s = radius_f[sorter]
    Rho_s = Rho_f[sorter]

    if fixed_radius is None:
        target_density = Rho_s[0] * 0.01
        Rho_mavg = np.convolve(Rho_s, np.ones(
            MOVING_AVERAGE_WINDOW), mode='valid') / MOVING_AVERAGE_WINDOW
        boundary_arg = np.argmax(
            Rho_mavg < target_density) + MOVING_AVERAGE_WINDOW
    else:
        boundary_arg = np.argmax(radius_s >= fixed_radius) + MOVING_AVERAGE_WINDOW
    boundary_range = slice(boundary_arg - MOVING_AVERAGE_WINDOW, boundary_arg)
    selected_range = slice(0, boundary_arg)
    boundary_radius = radius_s[boundary_arg]
    if (fixed_radius is None): print(f"selection radius = {boundary_radius}")
    if boundary_arg < MOVING_AVERAGE_WINDOW * 2:
        print(f"warn: selected volume is small. boundary_arg={boundary_arg}")
    space_sorted = [sp.flatten()[sorter] for sp in Space3]
    boundary_positions = tuple([s[boundary_range] for s in space_sorted])
    selected_Psi = np.stack(
        [ps.flatten()[sorter][selected_range] for ps in Psi3], axis=0)
    return boundary_radius, boundary_positions, selected_Psi


In [ ]:
Psi = all_Psis[6][0][-1]

In [ ]:
def plot_rho_over_r(Psi3):
    Rho = get_density(Psi3)
    Rho_f = Rho.flatten()
    densest_point_idx = np.unravel_index(np.argmax(Rho_f), shape=Rho.shape)
    densest_point = tuple([sp[densest_point_idx] for sp in Space3])
    radius = get_radius_grid(box_N, box_dx, densest_point)
    radius_f = radius.flatten()
    sorter = np.argsort(radius_f)
    radius_s = radius_f[sorter]
    Rho_s = Rho_f[sorter]
    counts_we, edges = np.histogram(
        radius_s, weights=Rho_s, bins=np.linspace(1, 200, 400))
    counts_uw, _ = np.histogram(radius_s, bins=np.linspace(1, 200, 400))
    y = counts_we/np.maximum(counts_uw, 1)
    x = edges
    # cond = radius_s < 8
    # x = radius_s[cond]
    # y = Rho_s[cond]
    return pgo.Scatter(x=x, y=y, mode='markers')
fig = pgo.Figure(data=plot_rho_over_r(Psi))
fig.update_yaxes(type="log")
fig.update_xaxes(type="log")
pass
fig.write_html("2022-07-30_Rho_over_r.html")


In [ ]:
fig.show()

In [ ]:
radius, boundary_points, selected_Psi = find_dense_circle(Psi)
(bx, by, bz) = boundary_points
plots = [
	[plot_snap_density(Psi)],
]
fig, js = plot_multiple_3Ds(plots)
fig.add_trace(pgo.Mesh3d(
	x=bx, y=by, z=bz,
	opacity=0.4, color='red', alphahull=0,
), 1, 1)
fig.update_layout(
	coloraxis = {
		'colorscale': 'viridis',
		'cmin': 0.0,
		'cmax': 1.0,
	},
)
fig.write_html("2022-07-30-core.html", post_script=js)
pass

In [ ]:
from multiprocessing import Pool
def spin_norm_over_mass(Psi):
	norm = np.sqrt(np.sum(np.square(
		np.sum(get_spins(Psi), axis=tuple(range(1,Psi.ndim)))
	)))
	mass = np.sum(get_density(Psi))
	return norm / mass
def do_one(group):
	core_spins = np.zeros(3)
	full_spins = np.zeros(3)
	for Psi in group:
		radius, boundary_points, selected_Psi = find_dense_circle(Psi, fixed_radius=3, use_gp=True)
		cs = np.sum(get_spins(selected_Psi), axis=(1,))
		core_spins += cs / np.sum(get_density(selected_Psi))
		full_spins += np.sum(get_spins(Psi), axis=tuple(range(1, Psi.ndim))) / np.sum(get_density(Psi))
		# x.append(spin_norm_over_mass(Psi))
		# y.append(spin_norm_over_mass(selected_Psi))
		# gx += spin_norm_over_mass(Psi)
		# gy += spin_norm_over_mass(selected_Psi)
	# hovertexts.append(label)
	core_v = np.sqrt(np.sum(np.square(core_spins)))
	full_v = np.sqrt(np.sum(np.square(full_spins)))
	return (full_v / len(group), core_v / len(group))


def plot_Spin_norm_over_Rho(Psis):
	x = []
	y = []
	hovertexts = []
	with Pool(8) as p:
		res = p.map(do_one, Psis)
		for (rx, ry) in res:
			x.append(rx)
			y.append(ry)
	# for group in tqdm(Psis):
	# 	core_spins = np.zeros(3)
	# 	full_spins = np.zeros(3)

	# 	for Psi in group:
	# 		radius, boundary_points, selected_Psi = find_dense_circle(Psi, fixed_radius=3, use_gp=True)
	# 		cs = np.sum(get_spins(selected_Psi), axis=(1,))
	# 		core_spins += cs / np.sum(get_density(selected_Psi))
	# 		full_spins += np.sum(get_spins(Psi), axis=tuple(range(1, Psi.ndim))) / np.sum(get_density(Psi))
	# 		# x.append(spin_norm_over_mass(Psi))
	# 		# y.append(spin_norm_over_mass(selected_Psi))
	# 		# gx += spin_norm_over_mass(Psi)
	# 		# gy += spin_norm_over_mass(selected_Psi)
	# 	# hovertexts.append(label)
	# 	core_v = np.sqrt(np.sum(np.square(core_spins)))
	# 	full_v = np.sqrt(np.sum(np.square(full_spins)))
	# 	x.append(full_v / len(group))
	# 	y.append(core_v / len(group))
	return pgo.Scatter(
		x=x, y=y,
		hovertext=hovertexts,
		mode="markers"
	)


In [ ]:

import itertools
INTR_MODES_NEW = [
	# filename,		 label,				 color
	("nosi",		"No SI",				"#1abc9c"),
	("attractive",	"Strong Attractive SI",	"#8e44ad"),
	("repulsive",	"Strong Repulsive SI",	"#e74c3c"),
]
fig = make_subplots(1, 1)
all_Psis_grouped = [[sim[i] for sim in all_Psis] for i in range(len(INTR_MODES))]
# all_Psis_new_grouped = [[sim[i] for sim in all_Psis_new] for i in range(len(INTR_MODES_NEW))]

# for ((_, label, color), simuls) in itertools.chain(zip(INTR_MODES, all_Psis_grouped), zip(INTR_MODES_NEW, all_Psis_new_grouped)):
for ((_, label, color), simuls) in tqdm(zip(INTR_MODES, all_Psis_grouped)):
	trace = plot_Spin_norm_over_Rho([[sn for (ts,sn) in zip(PROGRESS_STEPS, simul)] for (n,simul) in enumerate(simuls, start=1) ])
	trace.marker.color = color	
	trace.name = label
	fig.add_trace(trace, 1, 1)
fig = pgo.Figure(fig)
fig.update_layout(yaxis_range=[0.0, 1.0], xaxis_range=[0.0, 1.0])
fig.write_html("2022-07-30-total-vs-core-r3.5-gp-averaged.html")

In [ ]:
print(selected_Psi.shape)
print(np.sum(get_spins(selected_Psi), axis=1).shape)

In [ ]:
print(spin_norm_over_mass(Psi))
print(spin_norm_over_mass(selected_Psi))
print(np.average(get_density(Psi)))
print(np.average(get_density(selected_Psi)))


In [ ]:
sim_paths = [f"../outputs/2022-07-30/8-solitons-random-128-nosi-run-{i}/" for i in range(1, 1+NUM_SIMULATIONS)]
configs = []
for sim_path in sim_paths:
	config = spio.loadmat(os.path.join(sim_path, "simConfig.mat"))
	configs.append(config)

In [ ]:
sizes = [config['simConfig']['sizes'][0][0] for config in configs]

In [ ]:
[np.max(s) for s in sizes]